<a href="https://colab.research.google.com/github/elangbijak4/blockchain-codes-use-generative-AI/blob/main/Demo_DeFi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import hashlib
import json
from time import time
from typing import List, Dict, Any

In [22]:
class Block:
    def __init__(self, index: int, previous_hash: str, timestamp: float, data: str, hash: str):
        self.index = index
        self.previous_hash = previous_hash
        self.timestamp = timestamp
        self.data = data
        self.hash = hash

In [23]:
class Blockchain:
    def __init__(self):
        self.chain: List[Block] = []
        self.create_genesis_block()

    def create_genesis_block(self):
        genesis_block = self.create_block(data="Genesis Block", previous_hash="0")
        self.chain.append(genesis_block)

    def create_block(self, data: str, previous_hash: str) -> Block:
        index = len(self.chain)
        timestamp = time()
        hash = self.hash_block(index, previous_hash, timestamp, data)
        return Block(index, previous_hash, timestamp, data, hash)

    def hash_block(self, index: int, previous_hash: str, timestamp: float, data: str) -> str:
        block_string = json.dumps({
            "index": index,
            "previous_hash": previous_hash,
            "timestamp": timestamp,
            "data": data
        }, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()

    def add_block(self, data: str):
        previous_block = self.chain[-1]
        new_block = self.create_block(data, previous_block.hash)
        self.chain.append(new_block)

    def is_chain_valid(self) -> bool:
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i - 1]
            if current_block.hash != self.hash_block(current_block.index, current_block.previous_hash, current_block.timestamp, current_block.data):
                return False
            if current_block.previous_hash != previous_block.hash:
                return False
        return True

    def display_chain(self):
        for block in self.chain:
            print(f"Block {block.index}:")
            print(f"  Previous Hash: {block.previous_hash}")
            print(f"  Timestamp: {block.timestamp}")
            print(f"  Data: {block.data}")
            print(f"  Hash: {block.hash}\n")

In [24]:
# ERC-20 Token Implementation
class ERC20Token:
    def __init__(self, name: str, symbol: str, initial_supply: int):
        self.name = name
        self.symbol = symbol
        self.total_supply = initial_supply
        self.balances: Dict[str, int] = {}
        self.allowances: Dict[str, Dict[str, int]] = {}

    def balance_of(self, owner: str) -> int:
        return self.balances.get(owner, 0)

    def transfer(self, sender: str, recipient: str, amount: int) -> bool:
        if self.balances.get(sender, 0) >= amount:
            self.balances[sender] = self.balances.get(sender, 0) - amount
            self.balances[recipient] = self.balances.get(recipient, 0) + amount
            return True
        return False

    def approve(self, owner: str, spender: str, amount: int) -> bool:
        if owner not in self.allowances:
            self.allowances[owner] = {}
        self.allowances[owner][spender] = amount
        return True

    def transfer_from(self, spender: str, sender: str, recipient: str, amount: int) -> bool:
        if self.allowances.get(sender, {}).get(spender, 0) >= amount and self.balances.get(sender, 0) >= amount:
            self.balances[sender] = self.balances.get(sender, 0) - amount
            self.balances[recipient] = self.balances.get(recipient, 0) + amount
            self.allowances[sender][spender] -= amount
            return True
        return False

In [25]:
# ERC-721 Token Implementation
class ERC721Token:
    def __init__(self, name: str, symbol: str):
        self.name = name
        self.symbol = symbol
        self.token_owner: Dict[int, str] = {}
        self.token_approvals: Dict[int, str] = {}
        self.owner_tokens: Dict[str, List[int]] = {}
        self.next_token_id = 1

    def balance_of(self, owner: str) -> int:
        return len(self.owner_tokens.get(owner, []))

    def owner_of(self, token_id: int) -> str:
        return self.token_owner.get(token_id, None)

    def mint(self, owner: str) -> int:
        token_id = self.next_token_id
        self.next_token_id += 1
        self.token_owner[token_id] = owner
        if owner not in self.owner_tokens:
            self.owner_tokens[owner] = []
        self.owner_tokens[owner].append(token_id)
        return token_id

    def transfer(self, sender: str, recipient: str, token_id: int) -> bool:
        if self.token_owner.get(token_id) == sender:
            self.token_owner[token_id] = recipient
            self.owner_tokens[sender].remove(token_id)
            if recipient not in self.owner_tokens:
                self.owner_tokens[recipient] = []
            self.owner_tokens[recipient].append(token_id)
            return True
        return False

    def approve(self, owner: str, approved: str, token_id: int) -> bool:
        if self.token_owner.get(token_id) == owner:
            self.token_approvals[token_id] = approved
            return True
        return False

    def transfer_from(self, approved: str, sender: str, recipient: str, token_id: int) -> bool:
        if self.token_approvals.get(token_id) == approved and self.token_owner.get(token_id) == sender:
            self.token_owner[token_id] = recipient
            self.owner_tokens[sender].remove(token_id)
            if recipient not in self.owner_tokens:
                self.owner_tokens[recipient] = []
            self.owner_tokens[recipient].append(token_id)
            del self.token_approvals[token_id]
            return True
        return False

In [26]:
# DeFi Lending/Borrowing Platform
class DeFiPlatform:
    def __init__(self, token: ERC20Token):
        self.token = token
        self.deposits: Dict[str, int] = {}
        self.loans: Dict[str, int] = {}

    def deposit(self, user: str, amount: int):
        if self.token.transfer(user, self, amount):
            self.deposits[user] = self.deposits.get(user, 0) + amount
            return True
        return False

    def withdraw(self, user: str, amount: int):
        if self.deposits.get(user, 0) >= amount:
            self.deposits[user] -= amount
            if self.token.transfer(self, user, amount):
                return True
            else:
                self.deposits[user] += amount  # Rollback on failure
        return False

    def borrow(self, user: str, amount: int):
        if self.token.transfer(self, user, amount):
            # Initialize the loan amount if the user is not in the loans dictionary
            if user not in self.loans:
                self.loans[user] = 0
            self.loans[user] += amount  # Increment the loan amount
            return True
        return False

    def repay(self, user: str, amount: int):
        if self.token.transfer(user, self, amount):
            self.loans[user] -= amount
            return True
        return False

In [27]:
# Demo
blockchain = Blockchain()
blockchain.add_block("First block after Genesis")
blockchain.add_block("Second block after Genesis")
blockchain.add_block("Third block after Genesis")

print("Blockchain valid:", blockchain.is_chain_valid())
blockchain.display_chain()

Blockchain valid: True
Block 0:
  Previous Hash: 0
  Timestamp: 1719212848.421989
  Data: Genesis Block
  Hash: 2a60682f44b6ee5f46fa30ee03e054885da94d4610f02a08e42ced63de6173ad

Block 1:
  Previous Hash: 2a60682f44b6ee5f46fa30ee03e054885da94d4610f02a08e42ced63de6173ad
  Timestamp: 1719212848.4222012
  Data: First block after Genesis
  Hash: 845baddc357a8d050b022bf594f47fad8d90cd7d9e6d3bd62b173673ce254643

Block 2:
  Previous Hash: 845baddc357a8d050b022bf594f47fad8d90cd7d9e6d3bd62b173673ce254643
  Timestamp: 1719212848.4223008
  Data: Second block after Genesis
  Hash: cc2e1a6430269d67ff3a450d4de27ca80e7f5974210dcd7eaa3f18a138e07404

Block 3:
  Previous Hash: cc2e1a6430269d67ff3a450d4de27ca80e7f5974210dcd7eaa3f18a138e07404
  Timestamp: 1719212848.422376
  Data: Third block after Genesis
  Hash: 1589b185b0b7f8f4b362d4fe6c2ab73b8606c657465dc39ecc3f6df1a1316b01



In [28]:
# Demo ERC-20
erc20 = ERC20Token(name="DemoToken", symbol="DT", initial_supply=1000)
erc20.balances["alice"] = 500
erc20.balances["bob"] = 500

print(f"Alice's balance: {erc20.balance_of('alice')}")
print(f"Bob's balance: {erc20.balance_of('bob')}")

erc20.transfer("alice", "bob", 100)
print(f"Alice's balance after transfer: {erc20.balance_of('alice')}")
print(f"Bob's balance after transfer: {erc20.balance_of('bob')}")

Alice's balance: 500
Bob's balance: 500
Alice's balance after transfer: 400
Bob's balance after transfer: 600


In [29]:
print("Blockchain valid:", blockchain.is_chain_valid())
blockchain.display_chain()

Blockchain valid: True
Block 0:
  Previous Hash: 0
  Timestamp: 1719212848.421989
  Data: Genesis Block
  Hash: 2a60682f44b6ee5f46fa30ee03e054885da94d4610f02a08e42ced63de6173ad

Block 1:
  Previous Hash: 2a60682f44b6ee5f46fa30ee03e054885da94d4610f02a08e42ced63de6173ad
  Timestamp: 1719212848.4222012
  Data: First block after Genesis
  Hash: 845baddc357a8d050b022bf594f47fad8d90cd7d9e6d3bd62b173673ce254643

Block 2:
  Previous Hash: 845baddc357a8d050b022bf594f47fad8d90cd7d9e6d3bd62b173673ce254643
  Timestamp: 1719212848.4223008
  Data: Second block after Genesis
  Hash: cc2e1a6430269d67ff3a450d4de27ca80e7f5974210dcd7eaa3f18a138e07404

Block 3:
  Previous Hash: cc2e1a6430269d67ff3a450d4de27ca80e7f5974210dcd7eaa3f18a138e07404
  Timestamp: 1719212848.422376
  Data: Third block after Genesis
  Hash: 1589b185b0b7f8f4b362d4fe6c2ab73b8606c657465dc39ecc3f6df1a1316b01



In [30]:
# Demo ERC-721
erc721 = ERC721Token(name="DemoNFT", symbol="DNFT")
token_id1 = erc721.mint("alice")
token_id2 = erc721.mint("bob")

print(f"Alice's balance of NFTs: {erc721.balance_of('alice')}")
print(f"Bob's balance of NFTs: {erc721.balance_of('bob')}")

erc721.transfer("alice", "bob", token_id1)
print(f"Alice's balance of NFTs after transfer: {erc721.balance_of('alice')}")
print(f"Bob's balance of NFTs after transfer: {erc721.balance_of('bob')}")

Alice's balance of NFTs: 1
Bob's balance of NFTs: 1
Alice's balance of NFTs after transfer: 0
Bob's balance of NFTs after transfer: 2


# DeFi: Platform Pinjam Meminjam (Lending/Borrowing)

In [31]:
# Demo DeFi Platform
defi_platform = DeFiPlatform(token=erc20)
defi_platform.token.balances[defi_platform] = 0

print("DeFi Platform balances:")
print(f" Alice: {defi_platform.token.balance_of('alice')}")
print(f" Bob: {defi_platform.token.balance_of('bob')}")
print(f" DeFi Platform: {defi_platform.token.balance_of(defi_platform)}")

print("\nAlice deposits 200 tokens into DeFi Platform")
defi_platform.deposit("alice", 200)
print("DeFi Platform balances after deposit:")
print(f" Alice: {defi_platform.token.balance_of('alice')}")
print(f" DeFi Platform: {defi_platform.token.balance_of(defi_platform)}")

print("\nBob borrows 100 tokens from DeFi Platform")
defi_platform.borrow("bob", 100)
print("DeFi Platform balances after borrowing:")
print(f" Bob: {defi_platform.token.balance_of('bob')}")
print(f" DeFi Platform: {defi_platform.token.balance_of(defi_platform)}")

print("\nBob repays 100 tokens to DeFi Platform")
defi_platform.repay("bob", 100)
print("DeFi Platform balances after repayment:")
print(f" Bob: {defi_platform.token.balance_of('bob')}")
print(f" DeFi Platform: {defi_platform.token.balance_of(defi_platform)}")

print("\nAlice withdraws 200 tokens from DeFi Platform")
defi_platform.withdraw("alice", 200)
print("DeFi Platform balances after withdrawal:")
print(f" Alice: {defi_platform.token.balance_of('alice')}")
print(f" DeFi Platform: {defi_platform.token.balance_of(defi_platform)}")

DeFi Platform balances:
 Alice: 400
 Bob: 600
 DeFi Platform: 0

Alice deposits 200 tokens into DeFi Platform
DeFi Platform balances after deposit:
 Alice: 200
 DeFi Platform: 200

Bob borrows 100 tokens from DeFi Platform
DeFi Platform balances after borrowing:
 Bob: 700
 DeFi Platform: 100

Bob repays 100 tokens to DeFi Platform
DeFi Platform balances after repayment:
 Bob: 600
 DeFi Platform: 200

Alice withdraws 200 tokens from DeFi Platform
DeFi Platform balances after withdrawal:
 Alice: 400
 DeFi Platform: 0


# Bursa Terdesentralisasi (Decentralized Exchange, DEX)

In [32]:
# DeFi Decentralized Exchange (DEX) Platform
class DeFiDEX:
    def __init__(self, token1: ERC20Token, token2: ERC20Token):
        self.token1 = token1
        self.token2 = token2
        self.liquidity_pool1: Dict[str, int] = {}
        self.liquidity_pool2: Dict[str, int] = {}
        self.total_liquidity_token1 = 0
        self.total_liquidity_token2 = 0

    def add_liquidity(self, user: str, amount1: int, amount2: int):
        if self.token1.transfer(user, self, amount1) and self.token2.transfer(user, self, amount2):
            self.liquidity_pool1[user] = self.liquidity_pool1.get(user, 0) + amount1
            self.liquidity_pool2[user] = self.liquidity_pool2.get(user, 0) + amount2
            self.total_liquidity_token1 += amount1
            self.total_liquidity_token2 += amount2
            return True
        return False

    def remove_liquidity(self, user: str, amount1: int, amount2: int):
        if self.liquidity_pool1.get(user, 0) >= amount1 and self.liquidity_pool2.get(user, 0) >= amount2:
            self.liquidity_pool1[user] -= amount1
            self.liquidity_pool2[user] -= amount2
            self.total_liquidity_token1 -= amount1
            self.total_liquidity_token2 -= amount2
            if self.token1.transfer(self, user, amount1) and self.token2.transfer(self, user, amount2):
                return True
            else:
                self.liquidity_pool1[user] += amount1  # Rollback on failure
                self.liquidity_pool2[user] += amount2  # Rollback on failure
                self.total_liquidity_token1 += amount1
                self.total_liquidity_token2 += amount2
        return False

    def swap(self, user: str, from_token: str, amount: int):
        if from_token == self.token1.symbol:
            if self.total_liquidity_token1 == 0:  # Check if token1 liquidity is zero
                return False
            required_amount2 = amount * self.total_liquidity_token2 / self.total_liquidity_token1
            if self.token1.transfer(user, self, amount) and self.token2.transfer(self, user, required_amount2):
                return True
        elif from_token == self.token2.symbol:
            if self.total_liquidity_token2 == 0:  # Check if token2 liquidity is zero
                return False
            required_amount1 = amount * self.total_liquidity_token1 / self.total_liquidity_token2
            if self.token2.transfer(user, self, amount) and self.token1.transfer(self, user, required_amount1):
                return True
        return False

In [33]:
# Demo DeFi DEX Platform
tokenA = ERC20Token(name="TokenA", symbol="TKA", initial_supply=1000)
tokenB = ERC20Token(name="TokenB", symbol="TKB", initial_supply=1000)
tokenA.balances["alice"] = 500
tokenB.balances["bob"] = 500

dex_platform = DeFiDEX(token1=tokenA, token2=tokenB)
dex_platform.token1.balances[dex_platform] = 0
dex_platform.token2.balances[dex_platform] = 0

print("\nAlice adds liquidity to DEX (100 TKA, 200 TKB)")
dex_platform.add_liquidity("alice", 100, 200)
print(f"DEX Liquidity Pool (TokenA): {dex_platform.total_liquidity_token1}")
print(f"DEX Liquidity Pool (TokenB): {dex_platform.total_liquidity_token2}")

print("\nBob swaps 50 TKB for TKA")
dex_platform.swap("bob", "TKB", 50)
print(f"Bob's balance of TKA after swap: {tokenA.balance_of('bob')}")
print(f"Bob's balance of TKB after swap: {tokenB.balance_of('bob')}")


Alice adds liquidity to DEX (100 TKA, 200 TKB)
DEX Liquidity Pool (TokenA): 0
DEX Liquidity Pool (TokenB): 0

Bob swaps 50 TKB for TKA
Bob's balance of TKA after swap: 0
Bob's balance of TKB after swap: 500


In [35]:
# DeFi Staking Platform
class DeFiStaking:
    def __init__(self, staking_token: ERC20Token, reward_token: ERC20Token, reward_rate: float):
        self.staking_token = staking_token
        self.reward_token = reward_token
        self.reward_rate = reward_rate
        self.stakes: Dict[str, int] = {}
        self.rewards: Dict[str, float] = {}
        self.last_update_time: Dict[str, float] = {}

    def _update_rewards(self, user: str):
        if user in self.stakes:
            time_now = time()
            staked_time = time_now - self.last_update_time.get(user, time_now)
            self.rewards[user] = self.rewards.get(user, 0.0) + staked_time * self.stakes[user] * self.reward_rate
            self.last_update_time[user] = time_now

    def stake(self, user: str, amount: int):
        self._update_rewards(user)
        if self.staking_token.transfer(user, self, amount):
            self.stakes[user] = self.stakes.get(user, 0) + amount
            return True
        return False

    def unstake(self, user: str, amount: int):
        self._update_rewards(user)
        if self.stakes.get(user, 0) >= amount:
            self.stakes[user] -= amount
            if self.staking_token.transfer(self, user, amount):
                return True
            else:
                self.stakes[user] += amount  # Rollback on failure
        return False

    def claim_rewards(self, user: str):
        self._update_rewards(user)
        reward = self.rewards.get(user, 0.0)
        if reward > 0:
            self.rewards[user] = 0.0
            self.reward_token.balances[self] -= reward
            self.reward_token.balances[user] += reward
            return reward
        return 0.0

In [36]:
# Demo DeFi Staking Platform
reward_token = ERC20Token(name="RewardToken", symbol="RTK", initial_supply=1000)

staking_platform = DeFiStaking(staking_token=tokenA, reward_token=reward_token, reward_rate=0.1)
reward_token.balances[staking_platform] = 1000 # Now this line should work without error

print("\nAlice stakes 100 TKA into Staking Platform")
staking_platform.stake("alice", 100)
print(f"Alice's staked balance: {staking_platform.stakes['alice']} TKA")

# Simulate time passing for rewards
import time as t
t.sleep(2)  # Simulate 2 seconds passing

print("\nAlice claims rewards from Staking Platform")
staking_rewards = staking_platform.claim_rewards("alice")
print(f"Alice's staking rewards: {staking_rewards} RTK")


Alice stakes 100 TKA into Staking Platform
Alice's staked balance: 100 TKA

Alice claims rewards from Staking Platform
Alice's staking rewards: 0.0 RTK
